In [1]:
%use krangl


In [2]:
// %use kravis
@file:DependsOn("com.github.holgerbrandl:kravis:0.6.1")

In [3]:
@file:Repository("*mavenLocal")
@file:DependsOn("org.kalasim:kalasim:0.5-SNAPSHOT")

In [4]:
import krangl.*

import org.kalasim.*

import org.kalasim.misc.repeat
import org.koin.core.component.get


## Entity Definition

In [5]:
// Define simulation types
class Fork : Resource()

class Philosopher(name: String, val leftFork: Fork, val rightFork: Fork) : Component(name) {
    val thinking = exponential(1)
    val eating = exponential(1)

    override fun process() = sequence {
        while(true) {
            hold(thinking())
            request(leftFork) {
                hold(0.1) // wait before taking the second fork
                request(rightFork) {
                    hold(eating())
                    log("$name is eating")
                }
            }
        }
    }
}

## Simulation

In [7]:
// run simulation
val sim = createSimulation {
    traceCollector()

    // create forks and resources
    val names = listOf("Socrates", "Pythagoras", "Plato", "Aristotle")
    val forks = repeat(names.size) { Fork() }.repeat().take(names.size + 1).toList()
    names.forEachIndexed { idx, name ->
        Philosopher(name, forks[idx], forks[(idx + 1).rem(forks.size)])
    }

    run(100)
}

## Analysis

In [ ]:
data class RequestRecord(val requester: String, val timestamp: Double, val resource: String, val quantity: Double)

val tc = sim.get<TraceCollector>()
val requests = tc.filterIsInstance<ResourceEvent>().map {
    val amountDirected = (if(it.type == ResourceEventType.RELEASED) -1 else 1) * it.amount
    RequestRecord(it.requester.name, it.time, it.resource.name, amountDirected)
}

val requestsDf = requests.asDataFrame()
    .groupBy("requester")
    .sortedBy("requester", "timestamp")
    .addColumn("end_time") { it["timestamp"].lag() }
    .filter { it["quantity"] gt 0 }

In [ ]:
 requestsDf.schema()

In [ ]:
requestsDf.head()

In [ ]:
import kravis.*


In [ ]:
SessionPrefs.OUTPUT_DEVICE = kravis.device.JupyterDevice()

In [ ]:
SessionPrefs.OUTPUT_DEVICE

In [ ]:
// try to visualize it with kravis first

requestsDf.plot(x="timestamp").geomHistogram().show()

// also see https://github.com/holgerbrandl/data_science_with_kotlin/blob/master/report_rendering/nyc/WildRideThroughNYC.ipynb


In [9]:
 requestsDf.schema()

DataFrame with 240 observations
quantity   [Dbl]  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...
requester  [Str]  Pythagoras, Pythagoras, Pythagoras, Pythagoras, Pythagoras, Pythagoras, Pythagoras, Pythagoras, Pyth...
resource   [Str]  Fork.2, Fork.3, Fork.2, Fork.3, Fork.2, Fork.3, Fork.2, Fork.3, Fork.2, Fork.3, Fork.2, Fork.3, Fork...
timestamp  [Dbl]  0.366, 0.466, 2.033, 5.903, 8.17, 11.108, 13.107, 14.125, 16.613, 17.834, 20.201, 21.242, 23.915, 26...
end_time   [Dbl]  <NA>, 0.366, 0.798, 2.033, 7.667, 8.17, 11.743, 13.107, 14.793, 16.613, 19.011, 20.201, 22.639, 23.9...


In [10]:
requestsDf.head()

Grouped by: *[requester]
A DataFrame: 5 x 5
    quantity    requester   resource   timestamp   end_time
1          1   Pythagoras     Fork.2       0.366       <NA>
2          1   Pythagoras     Fork.3       0.466      0.366
3          1   Pythagoras     Fork.2       2.033      0.798
4          1   Pythagoras     Fork.3       5.903      2.033
5          1   Pythagoras     Fork.2        8.17      7.667

In [11]:
import kravis.*


In [14]:
SessionPrefs.OUTPUT_DEVICE = kravis.device.JupyterDevice()

In [16]:
SessionPrefs.OUTPUT_DEVICE

kravis.device.JupyterDevice@6cf11b63

In [17]:
// try to visualize it with kravis first

requestsDf.plot(x="timestamp").geomHistogram().show()

// also see https://github.com/holgerbrandl/data_science_with_kotlin/blob/master/report_rendering/nyc/WildRideThroughNYC.ipynb


java.lang.UnsupportedOperationException
krangl.Extensions.rowData(Extensions.kt:902)
krangl.TableIOKt.writeCSV(TableIO.kt:297)
krangl.TableIOKt.writeTSV(TableIO.kt:274)
krangl.TableIOKt.writeTSV$default(TableIO.kt:273)
kravis.render.LocalR.render$kravis(LocalR.kt:17)
kravis.GGPlot.save(GGPlot2.kt:164)
kravis.device.JupyterDevice.show(JupyterDevice.kt:26)
kravis.device.JupyterDevice.show(JupyterDevice.kt:21)
kravis.GGPlot.show(GGPlot2.kt:169)
Line_101_jupyter.<init>(Line_101.jupyter.kts:3)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:96)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.

In [ ]:
val rand = java.util.Random()
val data = mapOf<String, Any>(
    "rating" to List(200) { rand.nextGaussian() } + List(200) { rand.nextGaussian() * 1.5 + 1.5 },
    "cond" to List(200) { "A" } + List(200) { "B" }
)

var p = lets_plot(data)
p += geom_density(color="dark_green", alpha=.3) {x="rating"; fill="cond"}
p + ggsize(500, 250) +  